In [47]:
activity_labels = {}

In [48]:
with open("../../open_data/HAPT Data Set/activity_labels.txt") as file :
    data = file.readlines()

In [49]:
data

['1 WALKING           \n',
 '2 WALKING_UPSTAIRS  \n',
 '3 WALKING_DOWNSTAIRS\n',
 '4 SITTING           \n',
 '5 STANDING          \n',
 '6 LAYING            \n',
 '7 STAND_TO_SIT      \n',
 '8 SIT_TO_STAND      \n',
 '9 SIT_TO_LIE        \n',
 '10 LIE_TO_SIT        \n',
 '11 STAND_TO_LIE      \n',
 '12 LIE_TO_STAND      \n']

In [50]:
for line in data :
    #print(line.split(" ")[1].rstrip())
    #activity_labels[int(line.split(" ")[0])] = line.split(" ")[1].rstrip()
    activity_labels[line.split(" ")[1].rstrip()] = int(line.split(" ")[0])

In [51]:
activity_labels

{'WALKING': 1,
 'WALKING_UPSTAIRS': 2,
 'WALKING_DOWNSTAIRS': 3,
 'SITTING': 4,
 'STANDING': 5,
 'LAYING': 6,
 'STAND_TO_SIT': 7,
 'SIT_TO_STAND': 8,
 'SIT_TO_LIE': 9,
 'LIE_TO_SIT': 10,
 'STAND_TO_LIE': 11,
 'LIE_TO_STAND': 12}

In [52]:
labels = {} # tuple, tuple

In [53]:
with open("../../open_data/HAPT Data Set/RawData/labels.txt") as file :
    data = file.readlines()

In [54]:
for line in data :
    line = line.rstrip()
    key = tuple(line.split(" ")[0:3])
    value = tuple(line.split(" ")[3:5])
    if key in labels :
        labels[key].append(value)
    else :
        labels[key] = [value]

In [73]:
labels[('20', '10', '1')]

KeyError: ('20', '10', '1')

In [56]:
import glob, csv
acc_files = glob.glob("../../open_data/HAPT Data Set/RawData/acc_*.txt")

In [61]:
"/".join(acc_files[0].split("/")[0:-1])

'../../open_data/HAPT Data Set/RawData'

In [28]:
"gyr" + "_" + "_".join(acc_files[0].split("/")[-1].split("_")[1:])

'gyr_exp01_user01.txt'

In [17]:
str_test.split('.')[-1]

'txt'

In [64]:
import csv

In [143]:
def gather_all_activity(activity_label = "WALKING", output_path="/Volumes/Samsung_T3/TomatoCrew/TomatoEmbedded/data") :
    
    # open each accelerometer and gyroscope raw data and combine both into one activity file.
    with open(output_path + "/" + activity_label + '.csv', 'w') as out_file:
        
        csv_writer = csv.writer(out_file, delimiter=' ')
        csv_writer.writerow("Timestamp[us] A_X[mg] A_Y[mg] A_Z[mg] G_X[mdps] G_Y[mdps] G_Z[mdps]".split(" "))
        
        out_acc_lines = []
        out_gyr_lines = []
        
        for acc_file in acc_files : 
            
            gyro_file = "/".join(acc_file.split("/")[0:-1]) + "/gyro" + "_" + "_".join(acc_files[0].split("/")[-1].split("_")[1:])
            raw_acc_file = open(acc_file)
            raw_gyr_file = open(gyro_file)
            
            acc_file_name = acc_file.split("/")[-1].split(".")[0]
            
            exp_id = str(int(acc_file_name.split("_")[1].split("exp")[1]))
            user_id = str(int(acc_file_name.split("_")[2].split("user")[1]))
            
            # loop over labels values for a specific activity
            if (exp_id, user_id, str(activity_labels[activity_label])) in labels :
                lines_to_read = labels[(exp_id, user_id, str(activity_labels[activity_label]))]
                #print(lines_to_read)
                
                for lines in lines_to_read :
                    start = int(lines[0])
                    end = int(lines[1])
                    
                    # accelerometer file
                    for i, line in enumerate(raw_acc_file) :
                        
                        if i >= start and i <= end :
                            out_acc_lines.append(line)
                            
                    # gyroscope file
                    for i, line in enumerate(raw_gyr_file) :
                        
                        if i >= start and i <= end :
                            out_gyr_lines.append(line)
            
            raw_acc_file.close()
            raw_gyr_file.close()
            
        for i, data in enumerate(out_acc_lines) :
            
            #csv_writer.writerow(",".join( out_acc_lines[i].split(" ") + out_gyr_lines[i].split(" ") ) )
            #csv_writer.writerow(",".join( out_acc_lines[i].split(" ") + out_gyr_lines[i].split(" ") ) )
            acc_temp = list(map(float, out_acc_lines[i].split(" ")))
            gyr_temp = list(map(float, out_gyr_lines[i].split(" ")))
            
            acc_temp = list(map(lambda x: int(1000 * x), acc_temp))
            gyr_temp = list(map(lambda x: int(x / 0.017453 * 1000), gyr_temp))
            
            csv_writer.writerow([i] + acc_temp + gyr_temp)
        print(len(out_acc_lines), len(out_gyr_lines))
        

In [148]:
gather_all_activity(activity_label = "STANDING")

70207 70207
